<a href="https://colab.research.google.com/github/eisbetterthanpi/transformer/blob/main/gpt_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title data
import requests
# url="https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt" # https://www.tensorflow.org/text/tutorials/text_generation
url="https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.train.txt" # train test valid # https://pytorch.org/text/stable/datasets.html#penntreebank
out=requests.get(url)
with open("data.txt", "wb") as f:
    f.write(out.content)
text = open("data.txt", 'rb').read().decode(encoding='utf-8')

url="https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.test.txt" # train test valid # https://pytorch.org/text/stable/datasets.html#penntreebank
out=requests.get(url)
with open("data_.txt", "wb") as f: f.write(out.content)
test_text = open("data_.txt", 'rb').read().decode(encoding='utf-8')

# print(len(text))
# print(text[000:1000])
# data = ''.join(text)
# chars = sorted(list(set(data)))
# print(chars)


In [ ]:
!pip install -qU datasets # restart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [1]:
# @title hf dataset
from datasets import load_dataset

# https://huggingface.co/datasets/Salesforce/wikitext
dataset = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1") # wikitext-103-raw-v1

text = dataset["train"]["text"]
test_text = dataset["test"]["text"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
# @title tiktoken dataloader
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import tiktoken # https://github.com/openai/tiktoken/tree/main

class CharDataset(Dataset): # https://github.com/karpathy/minGPT
    def __init__(self, raw_data, seq_len):
        data = ''.join(raw_data)
        self.enc = tiktoken.get_encoding("gpt2") # https://github.com/openai/tiktoken/blob/main/tiktoken/core.py
        self.vocab_size = self.enc.n_vocab # gpt2:50257
        self.data = self.data_process(data) # list of int
        self.seq_len = seq_len

    def data_process(self, data): # str 10780437
        return torch.tensor(self.enc.encode(data))

    def __len__(self):
        return len(self.data)//(self.seq_len+1)

    def __getitem__(self, idx):
        dix = self.data[idx*(self.seq_len+1) : (idx+1)*(self.seq_len+1)]
        x, y = dix[:-1], dix[1:]
        return x, y

seq_len = 100 # 128
train_data = CharDataset(text, seq_len) # one line of poem is roughly 50 characters
test_data = CharDataset(test_text, seq_len) # one line of poem is roughly 50 characters
from torch.utils.data.dataloader import DataLoader
batch_size = 64 #512
train_loader = DataLoader(train_data, shuffle=True, pin_memory=True, batch_size=batch_size, num_workers=2, drop_last=True) # num_workers = 4
test_loader = DataLoader(test_data, shuffle=True, pin_memory=True, batch_size=batch_size, num_workers=0, drop_last=True)

# https://github.com/openai/tiktoken/blob/main/tiktoken/core.py
def encode(context):
    if type(context) == str: return torch.tensor([train_loader.dataset.enc.encode(context)], device=device)
    elif type(context) == list: return train_loader.dataset.enc.encode_batch(context)
    else: raise Exception
def decode(x): return train_loader.dataset.enc.decode(list(x))
# for x,y in train_loader:
#     break



In [ ]:
# @title char dataloader
# https://github.com/Sam-Armstrong/tinyGPT/blob/main/Training.py
# https://colab.research.google.com/github/karpathy/minGPT/blob/master/play_char.ipynb
# https://github.com/karpathy/nanoGPT
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class CharDataset(Dataset): # https://github.com/karpathy/minGPT
    def __init__(self, raw_data, seq_len):
        data = ''.join(raw_data)
        chars = sorted(list(set(data)))
        self.vocab_size = len(chars) # 283
        self.stoi = {ch:i for i,ch in enumerate(chars)}
        self.itos = {i:ch for i,ch in enumerate(chars)}
        self.data = self.data_process(data) # list of int
        self.seq_len = seq_len

    def data_process(self, data): # str 10780437
        return torch.tensor([self.stoi.get(c) for c in data]) # list of int 4570571 # stoi.get(c,UNK_IDX)

    def __len__(self):
        # return len(self.data) - self.seq_len
        return len(self.data)//(self.seq_len+1)

    def __getitem__(self, idx):
        # dix = self.data[idx:idx + self.seq_len + 1]
        dix = self.data[idx*(self.seq_len+1) : (idx+1)*(self.seq_len+1)]
        x, y = dix[:-1], dix[1:]
        return x, y


seq_len = 100 # 128
train_data = CharDataset(text, seq_len) # one line of poem is roughly 50 characters
test_data = CharDataset(test_text, seq_len) # one line of poem is roughly 50 characters
from torch.utils.data.dataloader import DataLoader
batch_size = 64 #512
train_loader = DataLoader(train_data, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 2) # num_workers = 4
test_loader = DataLoader(test_data, shuffle = True, pin_memory = True, batch_size = batch_size, num_workers = 0)


def encode(context): return torch.tensor([train_data.stoi.get(c) for c in context], device=device).unsqueeze(0)
def decode(x): return ''.join([train_data.itos[int(i)] for i in x])
# for x,y in train_loader:
#     break


In [3]:
# @title RoPE
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class RoPE(nn.Module): # Rotary Positional Embeddings
    def __init__(self, dim, seq_len=512, base=10000):
        super().__init__()
        theta = 1.0 / (base ** (torch.arange(0, dim, step=2) / dim))
        pos = torch.arange(seq_len).unsqueeze(1)
        angles = (pos * theta).unsqueeze(-1) # [seq_len, 1] * [dim // 2] -> [seq_len, dim // 2, 1]
        self.rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1).to(device) # [seq_len, dim // 2, 2]

    def forward(self, x):
        batch, seq_len, dim = x.shape
        # if rot_emb.shape[0] < seq_len: self.__init__(dim, seq_len)
        rot_emb = self.rot_emb[:seq_len].unsqueeze(0).expand(batch, -1, -1, -1) # [batch, seq_len, dim//2, 2]
        x = x.reshape(batch, seq_len, dim // 2, 2)
        rot_x = x * rot_emb
        return rot_x.flatten(-2)

dim=16
seq_len=512
rope = RoPE(dim, seq_len, base=10000)
x = torch.rand(4,64,dim, device=device)
out = rope(x)

print(out.shape)


torch.Size([4, 64, 16])


In [4]:
# @title snake
# https://github.com/Aria-K-Alethia/BigCodec/blob/main/vq/activations.py
# https://github.com/zhenye234/X-Codec-2.0/blob/main/vq/activations.py#L62
# Implementation adapted from https://github.com/EdwardDixon/snake under the MIT license.
import torch
from torch import nn

# class Snake(nn.Module):
#     def __init__(self, in_features, alpha=1.0, alpha_logscale=False):
#         super().__init__()
#         # self.in_features = in_features
#         self.alpha_logscale = alpha_logscale
#         if self.alpha_logscale: # log scale alphas initialized to zeros
#             self.alpha = nn.Parameter(torch.zeros(in_features) * alpha)
#         else: # linear scale alphas initialized to ones
#             self.alpha = nn.Parameter(torch.ones(in_features) * alpha)

#     def forward(self, x):
#         alpha = self.alpha.unsqueeze(0).unsqueeze(-1) # line up with x to [B, C, T]
#         if self.alpha_logscale:
#             alpha = torch.exp(alpha)
#         x = x + (1.0 / (alpha + 1e-9)) * torch.pow(torch.sin(x * alpha), 2) # Snake ∶= x + 1/a * sin^2(ax)
#         return x


class Snake(nn.Module):
    def __init__(self, dim, *args, **kwargs):
        super().__init__()
        # self.alpha = nn.Parameter(torch.zeros(1,dim,1)).exp() # alpha_logscale=True
        self.alpha = nn.Parameter(torch.ones(1,1,dim)*1.) # 1.

    def forward(self, x):
        return x + (1.0 / (self.alpha + 1e-9)) * torch.pow(torch.sin(x * self.alpha), 2) # Snake ∶= x + 1/a * sin^2(ax)


@torch.jit.script
def snake(x, alpha): # [b,c,t], [1,c,1]
    return x + (alpha + 1e-9).reciprocal() * torch.sin(alpha * x).pow(2) # [b,c,t]

class Snake1d(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.alpha = nn.Parameter(torch.zeros(1,1,dim)).exp()
        self.alpha = nn.Parameter(torch.ones(1,1,dim)*1.) # 1.

    def forward(self, x):
        return snake(x, self.alpha)



class SnakeBeta(nn.Module):
    def __init__(self, dim, alpha=1.0, alpha_logscale=False):
        super().__init__()
        self.alpha_logscale = alpha_logscale
        self.alpha = nn.Parameter(torch.ones(1,1,dim) * (0. if self.alpha_logscale else alpha))
        self.beta = nn.Parameter(torch.ones(1,1,dim) * (0. if self.alpha_logscale else alpha))

    def forward(self, x): # [b,c,t]
        alpha = self.alpha#.unsqueeze(0).unsqueeze(-1) # line up with x to [B, C, T]
        beta = self.beta#.unsqueeze(0).unsqueeze(-1)
        if self.alpha_logscale:
            alpha = torch.exp(alpha)
            beta = torch.exp(beta)
        x = x + (1. / (beta + 1e-9)) * pow(torch.sin(x * alpha), 2) # SnakeBeta ∶= x + 1/b *sin^2(ax)
        return x # [b,c,t]

b,c,t = 5,16,7
# a1 = Snake(c)
a1 = Snake(c, alpha_logscale=True) # 70.4 µs 69.9
# a1 = Snake1d(c) # 47.8 µs 48.3
# a1 = SnakeBeta(256)
x = torch.randn(b,c,t)
# x = a1(x)
# print(x.shape)


In [5]:
# @title Swwish
import torch.nn as nn

# import math
# s = (2/math.pi)**.5
# a1=(2*s)**(1/2)
# a2=(4*s)**(1/4)
# a3=(8*s)**(1/6)
# 1/2(1+x-cos(x))
# 1/2(1+x-cos(a*x))

@torch.jit.script
def swwish(x): return .5 * (1 + x - x.cos())


@torch.jit.script
def learntswwish(x, alpha): # [b,c,t], [1,c,1]
    return .5 * (1 + x - torch.cos(alpha * x)) # [b,c,t]
    # return .5 * (1/alpha + x - torch.cos(alpha * x)/alpha) # [b,c,t]

class LearntSwwish(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.alpha = nn.Parameter(torch.zeros(1,dim,1)).exp()
        self.alpha = nn.Parameter(torch.ones(1,dim,1)*1.) # 1.

    def forward(self, x):
        return learntswwish(x, self.alpha)


class Swwish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        # return .5 * (1 + x - x.cos())
        return .5 * (1 + x - 1.25*x.cos())


In [6]:
# @title enc dec trans new
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def zero_module(module):
    for p in module.parameters():
        p.detach().zero_()
    return module

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads=None, d_head=8, cond_dim=None, dropout=0.): # .1
        super().__init__()
        self.d_model, self.n_heads, self.d_head = d_model, n_heads, d_model // n_heads
        self.cond_dim = cond_dim
        self.q = nn.Linear(d_model, d_model, bias=False)
        self.kv = nn.Linear(cond_dim or d_model, 2*d_model, bias=False)
        self.lin = zero_module(nn.Linear(d_model, d_model))
        self.drop = nn.Dropout(dropout) # indp before q,k,v; after linout
        self.scale = self.d_head**-.5

    def forward(self, x, cond=None, mask=None): # [batch, T, d_model]=[batch, h*w, c], [batch, num_tok, cond_dim], [batch,T]
        if self.cond_dim==None: cond=x # is self attn
        q = self.q(x).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2) # [batch, T, d_model] -> [batch, n_heads, T, d_head]
        # K = self.k(x).unflatten(-1, (self.n_heads, self.d_head)).transpose(1, 2)
        k, v = self.kv(cond).unflatten(-1, (self.n_heads, 2*self.d_head)).transpose(1, 2).chunk(2, dim=-1) # [batch, n_heads, T/num_tok, d_head]

        # # (quadratic) attention # Softmax(q @ k.T) @ v
        out = F.scaled_dot_product_attention(q, k, v, attn_mask=mask.unsqueeze(1) if mask != None else None, dropout_p=0) # mask: [batch,len_q, len_v]
        # out = F.scaled_dot_product_attention(q, k, v, is_causal=True, dropout_p=0) # mask: [batch,len_q, len_v]
        # attn = q @ k.transpose(-2,-1) * self.scale # [batch, n_heads, T] # [batch, n_heads, T, T/num_tok]
        # # if mask != None: attn = attn.masked_fill(mask[:, None, :, None], -torch.finfo(attn.dtype).max) # [batch,T]->[batch,1,T,1]
        # if mask != None: attn = attn.masked_fill(mask.unsqueeze(1), -torch.finfo(attn.dtype).max) # [b,t,t]->[b,1,t,t]
        # attention = torch.softmax(attn, dim=-1)
        # out = self.drop(attention) @ v # [batch, n_heads, T, d_head]

        out = out.transpose(1,2).flatten(2)
        return self.drop(self.lin(out)) # [batch, T, d_model]


class SwiGLU(nn.Module): # https://arxiv.org/pdf/2002.05202
    def __init__(self, d_model, ff_dim): # d_model * 3*ff_dim params
        super().__init__()
        self.lin0 = nn.Linear(d_model, 2*ff_dim, bias=False)
        self.lin1 = zero_module(nn.Linear(ff_dim, d_model, bias=False))

    def forward(self, x): # [b,t,d]
        x0, x1 = self.lin0(x).chunk(2, dim=-1)
        return self.lin1(x0*F.silu(x1))

# 2048*2
# 2048*7
# ff: d_model*2 *ff_dim params

class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, ff_dim, dropout=0):
        super().__init__()
        self.drop = nn.Dropout(dropout)
        self.norm = nn.RMSNorm(d_model)
        self.attn = MultiHeadAttention(d_model, n_heads, dropout=0)
        ff_dim = ff_dim or d_model*4
        # act = nn.ReLU() # ReLU SiLU GELU
        # act = Snake(ff_dim, alpha_logscale=True) # 70.4 µs 69.9
        # act = Snake1d(ff_dim) # 47.8 µs 48.3
        # act = SnakeBeta(ff_dim)
        # act = Swwish()
        # self.ff = nn.Sequential(
        #     nn.RMSNorm(d_model), nn.Linear(d_model, ff_dim), act,
        #     nn.RMSNorm(ff_dim), nn.Dropout(dropout), zero_module(nn.Linear(ff_dim, d_model))
        # )
        self.ff = SwiGLU(d_model, ff_dim)

    def forward(self, x, mask=None):
        x = x + self.drop(self.attn(self.norm(x), mask=mask))
        x = x + self.drop(self.ff(x))
        return x


# b,t,d = 2,5,16
# x = torch.rand(b,t,d)
# mask = torch.rand(b,t,t)>0
# model = EncoderLayer(d_model=d, n_heads=4, ff_dim=16)
# model = nn.Sequential(*[EncoderLayer(d_model=d, n_heads=4, ff_dim=16) for _ in range(2)])
# out =  model(x, mask)
# # out =  model(x)
# print(out.shape)


In [13]:
# @title GPT
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import inspect
class Seq(nn.Sequential):
    def __init__(self, *args):
        super().__init__(*args)
        for layer in self:
            params = inspect.signature(layer.forward).parameters.keys()
            layer._fwdparams = ','.join(params)

    def forward(self, x, mask=None):
        for layer in self:
            args = [x]
            if 'mask' in layer._fwdparams: args.append(mask)
            x = layer(*args)
        return x

class GPT(nn.Module):
    def __init__(self, in_dim, d_model=64, out_dim=None, n_heads=8, n_layers=1, ff_dim=None, dropout=0):
        super().__init__()
        self.d_model = d_model
        # self.pos_enc = PositionalEncoder(d_model, dropout=dropout)
        # self.pos_enc = LearntPosEnc(d_model, dropout=dropout)
        self.pos_enc = RoPE(d_model, base=10000)
        self.tok_emb = nn.Embedding(in_dim, d_model)
        self.encoder = Seq(*[EncoderLayer(d_model, n_heads, ff_dim, dropout) for _ in range(n_layers)])
        self.out = nn.Linear(d_model, out_dim)

    def forward(self, x, mask=None): # [b,t], [b,t,t]
        x = self.pos_enc(self.tok_emb(x))
        # print('gpt fwd', x.shape)
        x = self.encoder(x, mask)
        x = self.out(x)
        return x

input_size = num_classes = 50257 #train_data.vocab_size#50 # gpt2:50257
# print(train_data.vocab_size)
# model = GPT(input_size, d_model=64, out_dim=num_classes, n_layers=1).to(device)
model = GPT(input_size, d_model=512, out_dim=num_classes, n_layers=6).to(device)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
print(sum(p.numel() for p in model.encoder.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3)
# optim = torch.optim.AdamW(model.parameters(), 3e-4)

# batch 64, seqlen 100
# dim 64, nlyrs 1: 5.0gb
# 512, 1, 6.3, 95sec
# dim 512, 6 lyrs, 8.7 gb, 125 sec, 59393105 params
# dim 768, 12 lyrs, 12.4 gb, 257 sec, 110318161 params

# dim 64 ff 256, nlyrs 1, 5.0gb, ptb/wiki2 21/43 sec, 6548817 params
# dim 512 ff *4, nlyrs 6, 6.3gb, ptb/wiki2 21/43 sec, 6548817 params

# dim 512 ff *4, nlyrs 6, 9.2?gb, wiki2 348 sec, 76685393 params

x = torch.randint(0, input_size, (2, 5), device=device)
out = model(x)
print(out.shape)
# print(out)


76685393
25171968
torch.Size([2, 5, 50257])


In [8]:
# @title perplexity
import torch
import torch.nn as nn
import torch.nn.functional as F
# https://www.comet.com/site/blog/perplexity-for-llm-evaluation/

def Perplexity(logits, target): # [b,t,vocab_size], [b,t]
    log_probs = F.log_softmax(logits, dim=-1)
    nll = -log_probs.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1) # [b,t]
    perplexity = nll.mean().exp()
    return perplexity

# logits = torch.randn(2, 4, 10)
# target = torch.tensor([[1, 2, 3, 4], [4, 3, 2, 1]])

# perplexity = Perplexity(logits, target)
# # perplexity = Perplexity(logits[:,:-1], y[:,1:])
# print(f'Perplexity: {perplexity}')


In [ ]:
# @title train test generate
import torch
from torch.nn import functional as F
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.GradScaler()

def strain(model, dataloader, optimizer, scheduler=None): # train function with automatic mixed precision
    model.train()
    total_loss = 0
    for i, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        with torch.autocast(device_type=device, dtype=torch.bfloat16):
            causal_mask = torch.ones(x.size(1), x.size(1), dtype=bool, device=device).tril(diagonal=0).repeat(x.shape[0],1,1) # for F.scaled_dot_product_attention
            # causal_mask = ~torch.ones(x.size(1), x.size(1), dtype=bool, device=device).tril(diagonal=0).repeat(x.shape[0],1,1)
            logits = model(x, mask=causal_mask) #output = [batch size, trg len - 1, output dim]
            # logits, _ = model(x) # rnn
            loss = F.cross_entropy(logits.flatten(0,1), y.flatten()) # [b*t,d], [b*t]
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.unscale_(optim)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
        scaler.step(optimizer)
        scaler.update()
        # if scheduler is not None: scheduler.step()
        if i % (len(dataloader)//10) == 0: print("strain",loss.item())
        total_loss += loss.item()
        # try: wandb.log({"train loss": loss.item()/len(y)})
        try: wandb.log({"train loss": loss.item()})
        except NameError: pass
    return total_loss / len(dataloader)

def test(model, loader):
    model.eval()
    total_loss = 0
    for i, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            logits = model(x)
            # logits, _ = model(x) # rnn
            perplexity = Perplexity(logits[:,:-1], y[:,1:])
        loss = F.cross_entropy(logits.flatten(0,-2), y.flatten()) # [batch*seq_len, vocab_size], [batch*seq_len]
        total_loss+=loss.item()
        # if i % 100 == 0: print("test",loss.item())
        if i % 100 == 0: print("test",loss.item(),"ppty",perplexity.item())
        # try: wandb.log({"test loss": loss.item()/len(y)})
        try: wandb.log({"test loss": loss.item()})
        except NameError: pass
    return total_loss / len(loader)

def generate(model, context, max_steps=64, temperature=1):
    x = encode(context)#.to(device)
    model.eval()
    for n in range(max_steps):
        with torch.no_grad():
            output = model(x)
            # output, hidden = model(x, hidden) # rnn
        # print('generate', output.shape, hidden.shape)
        # hidden = hidden[:, -1, :].unsqueeze(1) # RNN/GRU
        output = output[:, -1] # get logit for last character
        output = output/temperature
        output = F.softmax(output, dim=-1) # vocab_size to char
        ix = torch.multinomial(output, num_samples=1) # rand sample by output distribution
        x = torch.cat((x, ix), dim=1)
    completion = decode(x.squeeze(0))
    return completion

import time
start = begin = time.time()
# for i in range(1):
for i in range(30):
    train_loss = strain(model, train_loader, optim, scheduler=None)
    test_loss = test(model, test_loader)
    print('Test Loss:', test_loss)
    print(generate(model, "this is what"))
    print(i, 'time:',time.time() - start, (time.time()-begin)/(i+1))
    start = time.time()


strain 10.91341781616211
strain 7.016169548034668
strain 6.5468645095825195
strain 6.219416618347168
strain 6.200089931488037
strain 5.935671806335449
strain 5.72556209564209
strain 5.687353610992432
strain 5.615914344787598
strain 5.5544328689575195
test 6.043789863586426 ppty 7036.03369140625
Test Loss: 5.976807771727096
this is what it as part of weapons into 68 . The mass and episodes to gather by part of whom he studied in tribute structures that the assassination bear that as Islamic competitiveness . Mynmann , who subsidiary of audio is towards , ignores , were described as he completed angel when a staco , a lack of the violate but asked Squad
0 time: 350.43189120292664 350.4318919181824
strain 5.009577751159668
strain 4.8494768142700195
strain 5.0731425285339355
strain 4.983185768127441
strain 4.875759124755859
strain 5.0334062576293945
strain 4.875674247741699
strain 4.817008018493652
strain 4.867505073547363
strain 5.014249801635742
test 5.752124786376953 ppty 8991.649414062

In [14]:
# @title wandb
!pip install -q wandb
import wandb # https://docs.wandb.ai/quickstart
wandb.login(key='487a2109e55dce4e13fc70681781de9f50f27be7')
try: run.finish()
except NameError: pass
run = wandb.init(project="gpt", config={"model": "res18",}) #

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


test loss,▆▅▆▇▃▆▆▆▇▇▆▆█▆▆█▆▇▆▄▂▃▂▂▃▂▄▂▃▂▂▄▁▁▂▂▁▂▂▃
train loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
test loss,6.05217
train loss,1.5716104253819703e+24


In [ ]:
# @title RNN pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
device = "cuda" if torch.cuda.is_available() else "cpu"

class RNN(nn.Module):
    def __init__(self, in_dim, d_model, out_dim=None, num_layers=1):
        super().__init__()
        if out_dim is None: out_dim = in_dim
        self.num_layers = num_layers
        self.d_model = d_model
        self.emb = nn.Embedding(in_dim, d_model)
        # self.rnn = nn.RNN(d_model, d_model, num_layers, batch_first=True)
        self.rnn = nn.GRU(d_model, d_model, num_layers, batch_first=True)
        # self.lstm = nn.LSTM(d_model, d_model, num_layers, batch_first=True)
        self.fc = nn.Linear(d_model, out_dim)

        # for p in self.parameters():
        #     if p.dim() > 1:
        #         nn.init.xavier_uniform_(p)

    def forward(self, x, hc=None): # lstm [batch_size, seq, in_dim]
        x = self.emb(x)
        if hc is None:
            h0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)
            c0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)
        else: h0,c0 = hc
        # print(x.shape, h0.shape,c0.shape)
        out, (h,c) = self.lstm(x, (h0,c0)) # [batch, seq_len, d_model], ([num_layers, batch, d_model] )
        # out = out[:, -1, :] # out: (n, 128)
        out = self.fc(out) # out: (n, 10)
        return out, (h, c)

    def forward(self, x, h=None): # rnn/gru
        x = self.emb(x)# * self.d_model**.5
        if h is None: h0 = torch.zeros((self.num_layers, x.size(0), self.d_model), device=device)
        else: h0 = h
        # print(x.shape, h0.shape)
        out, h = self.rnn(x, h0)
        # out = out[:, -1, :] # out: (n, 128)
        out = self.fc(out) # out: (n, 10)
        return out, h

hidden_size = 64 #128
num_layers = 1#2
input_size = num_classes = 50#train_data.vocab_size#65

model = RNN(input_size, hidden_size, num_classes, num_layers).to(device)
# print(model)
print(sum(p.numel() for p in model.parameters() if p.requires_grad)) # 19683
optim = torch.optim.AdamW(model.parameters(), 1e-3)

# 128,2
# Test Loss: 6.360389362062727
# this is what ween new york is well it a sign more directly into simply shares
# 0 time: 5.910429954528809 5.910431623458862
# 64,2
# Test Loss: 6.167358561924526
# this is what bull morp on its aftant opereals of a b. the hamally plans beces
# 0 time: 5.655158996582031 5.655160903930664
# 64,1
# Test Loss: 5.823708357129778
# this is what achan agrive
#  the guinesst on of promjects cl funds that jound
# 0 time: 4.788918495178223

# Test Loss: 8.247572830745153
# this is what that has months with unfinsings lide to N by well
#  next offited
# 29 time: 3.902247905731201 4.377542002995809

b,t=2,5

x = torch.randint(0,input_size, (b,t), device=device)
h = torch.rand(num_layers, b, hidden_size, device=device)
out, h = model(x, h)
print(out.shape, h.shape)


31410
torch.Size([2, 5, 50]) torch.Size([1, 2, 64])


In [ ]:
# @title mask translate
PAD_IDX=0
def make_src_mask(src):
    # return (src != PAD_IDX).unsqueeze(1).unsqueeze(2).to(device) # [batch_size, 1, 1, src_len]?
    return (src != PAD_IDX)[:,None,None,:].to(device) # [batch_size, 1, 1, src_len]?

# attn = attn.masked_fill(mask == 0, -1e10) # [batch, n_heads, seq_len, seq_len]
def make_trg_mask(trg):
    # trg_pad_mask = (trg != PAD_IDX).unsqueeze(1).unsqueeze(2).to(device)
    trg_pad_mask = (trg != PAD_IDX)[:,None,None,:].to(device) # [batch, 1, 1, trg_len]
    trg_len = trg.shape[1]
    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=device)).bool()
    # print('make_trg_mask', trg_pad_mask.shape, trg_sub_mask.shape) # [64, 1, 1, 10], [10, 10]
    trg_mask = trg_pad_mask & trg_sub_mask # [batch, 1, trg_len, trg_len]?
    return trg_mask

def translate(model, src_sentence):
    model.eval()
    src = de_transform(src_sentence).view(1,-1).to(device)
    num_tokens = src.shape[1]
    trg_indexes = [BOS_IDX]
    max_len = src.shape[1]+5
    for i in range(max_len):
        trg_tensor = torch.tensor(trg_indexes, dtype=torch.long, device=device).unsqueeze(0)
        src_mask, trg_mask = make_src_mask(src), make_trg_mask(trg_tensor)
        with torch.no_grad():
            output = model(src, trg_tensor, src_mask, trg_mask)
        pred_token = output.argmax(2)[:,-1].item() # batch_first=F -> ?
        trg_indexes.append(pred_token)
        if pred_token == EOS_IDX: break
    trg_tokens = torch.tensor(trg_indexes[1:-1]).flatten()
    return " ".join(en_vocab.lookup_tokens(list(trg_tokens.cpu().numpy())))

def translate_fast(model, src_sentence):
    model.eval()
    with torch.no_grad():
        src = de_transform(src_sentence).view(1,-1).to(device)
        num_tokens = src.shape[1]
        trg_indexes = [BOS_IDX]
        max_len = src.shape[1]+5
        src_mask = make_src_mask(src)
        output = model.encode(src, src_mask)
        for i in range(max_len):
            trg_tensor = torch.tensor(trg_indexes, dtype=torch.long, device=device).unsqueeze(0)
            trg_mask = make_trg_mask(trg_tensor)
            output = model.decode(src, trg_tensor, src_mask, trg_mask)
            pred_token = output.argmax(2)[:,-1].item() # batch_first=F -> ?
            trg_indexes.append(pred_token)
            if pred_token == EOS_IDX: break
        trg_tokens = torch.tensor(trg_indexes[1:-1]).flatten()
        return " ".join(en_vocab.lookup_tokens(list(trg_tokens.cpu().numpy())))

# UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # unknown, pad, bigining, end of sentence
# print(translate(model, "Eine Gruppe von Menschen steht vor einem Iglu ."))

src, trg = torch.randint(0, 100, (64, 10)), torch.randint(0, 100, (64, 10))
sm = make_src_mask(src)
tm = make_trg_mask(trg)
# print(sm.shape, tm.shape) # [64, 1, 1, 10], [64, 1, 10, 10]


torch.Size([64, 1, 1, 10]) torch.Size([64, 1, 10, 10])


In [ ]:
# @title translate train test

def train(model, dataloader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for src, trg in dataloader:
        src, trg = src.to(device), trg.to(device) #trg = [batch size, trg len]
        trg_input = trg[:,:-1]
        src_mask, trg_mask = make_src_mask(src), make_trg_mask(trg_input)
        print('train', src.shape, trg.shape, src_mask.shape, trg_mask.shape)
        output = model(src, trg_input, src_mask, trg_mask) #output = [batch size, trg len - 1, output dim]
        optimizer.zero_grad()
        loss = loss_fn(output.reshape(-1, output.shape[-1]), trg[:,1:].reshape(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(list(dataloader))

def test(model, dataloader, loss_fn):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for src, trg in dataloader:
            src, trg = src.to(device), trg.to(device) #trg = [batch size, trg len]
            trg_input = trg[:,:-1]
            src_mask, trg_mask = make_src_mask(src), make_trg_mask(trg_input)
            output = model(src, trg_input, src_mask, trg_mask) #output = [batch size, trg len - 1, output dim]
            loss = loss_fn(output.reshape(-1, output.shape[-1]), trg[:,1:].reshape(-1))
            epoch_loss += loss.item()
    return epoch_loss / len(list(dataloader))

# @title run
import time

loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_IDX)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9) # lr=0.0001

# for epoch in range(20):
for epoch in range(1):
    start_time = time.time()
    train_loss = train(model, train_loader, optimizer, loss_fn)
    val_loss = test(model, val_loader, loss_fn)
    end_time = time.time()
    print((f"Epoch: {epoch+1}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    # print(translate(model, "Eine Gruppe von Menschen steht vor einem Iglu .")) # A group of people standing in front of an igloo .
    # @title inference
    print(translate(model, "Eine Gruppe von Menschen steht vor einem Iglu .")) # A group of people stand in front of an igloo .
    print(translate(model, "Ein Koch in weißer Uniform bereitet Essen in einer Restaurantküche zu .")) # A chef in a white uniform prepares food in a restaurant kitchen .
    print(translate(model, "Zwei junge Mädchen spielen Fußball auf einem Feld. .")) # Two young girls play soccer on a field. .
    print(translate(model, "Eine Frau mit Hut und Sonnenbrille steht am Strand .")) # A woman wearing a hat and sunglasses stands on the beach .
    print(translate(model, "Zwei Freunde lachen und genießen ein Eis auf einer wunderschönen Wiese .")) # Two friends laugh and enjoy ice cream on a beautiful meadow .


Epoch: 1, Train loss: 5.402, Val loss: 4.186, Epoch time = 41.608s
A group of people are are are are are are in a .
Epoch: 2, Train loss: 3.898, Val loss: 3.545, Epoch time = 41.068s
A group of people are standing in a crowd of people .
Epoch: 3, Train loss: 3.353, Val loss: 3.125, Epoch time = 41.566s
A group of people standing in front of a crowd .
Epoch: 4, Train loss: 2.944, Val loss: 2.830, Epoch time = 40.756s
A group of people standing in front of a building .
Epoch: 5, Train loss: 2.630, Val loss: 2.596, Epoch time = 41.468s
A group of people standing in front of a crowd .
Epoch: 6, Train loss: 2.375, Val loss: 2.429, Epoch time = 41.023s
A group of people standing in front of a house .
Epoch: 7, Train loss: 2.166, Val loss: 2.307, Epoch time = 41.604s
A group of people stand in front of a house .
Epoch: 8, Train loss: 1.984, Val loss: 2.210, Epoch time = 40.876s
A group of people stand in front of an audience .
Epoch: 9, Train loss: 1.834, Val loss: 2.131, Epoch time = 41.496s

In [ ]:
'''
Epoch: 1, Train loss: 5.402, Val loss: 4.186, Epoch time = 41.608s
A group of people are are are are are are in a .
Epoch: 2, Train loss: 3.898, Val loss: 3.545, Epoch time = 41.068s
A group of people are standing in a crowd of people .
Epoch: 3, Train loss: 3.353, Val loss: 3.125, Epoch time = 41.566s
A group of people standing in front of a crowd .
Epoch: 4, Train loss: 2.944, Val loss: 2.830, Epoch time = 40.756s
A group of people standing in front of a building .
Epoch: 5, Train loss: 2.630, Val loss: 2.596, Epoch time = 41.468s
A group of people standing in front of a crowd .
Epoch: 6, Train loss: 2.375, Val loss: 2.429, Epoch time = 41.023s
A group of people standing in front of a house .
Epoch: 7, Train loss: 2.166, Val loss: 2.307, Epoch time = 41.604s
A group of people stand in front of a house .
Epoch: 8, Train loss: 1.984, Val loss: 2.210, Epoch time = 40.876s
A group of people stand in front of an audience .
Epoch: 9, Train loss: 1.834, Val loss: 2.131, Epoch time = 41.496s
A group of people are standing in front of an audience .
Epoch: 10, Train loss: 1.698, Val loss: 2.079, Epoch time = 41.052s
A group of people are standing in front of an empty house .
Epoch: 11, Train loss: 1.576, Val loss: 2.038, Epoch time = 41.570s
A group of people are standing in front of an audience .
Epoch: 12, Train loss: 1.475, Val loss: 2.033, Epoch time = 41.067s
A group of people stand in front of an audience .
Epoch: 13, Train loss: 1.381, Val loss: 2.017, Epoch time = 41.576s
A group of people stand in front of an operation .
Epoch: 14, Train loss: 1.292, Val loss: 1.977, Epoch time = 40.779s
A group of people stand in front of an operation .
Epoch: 15, Train loss: 1.213, Val loss: 1.948, Epoch time = 41.461s
A group of people stand in front of an operation .
Epoch: 16, Train loss: 1.139, Val loss: 1.940, Epoch time = 40.800s
A group of people stand in front of an igloo
Epoch: 17, Train loss: 1.073, Val loss: 1.940, Epoch time = 41.533s
A group of people stand in front of an igloo
Epoch: 18, Train loss: 1.006, Val loss: 1.939, Epoch time = 40.856s
A group of people stand in front of an igloo
Epoch: 19, Train loss: 0.944, Val loss: 1.947, Epoch time = 41.345s
A group of people stand in front of an igloo
Epoch: 20, Train loss: 0.893, Val loss: 1.956, Epoch time = 40.935s
A group of people stand in front of an igloo
'''

In [ ]:
# @title bleu
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    trgs = []
    pred_trgs = []
    for datum in data:
        src = vars(datum)['src']
        trg = vars(datum)['trg']
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        pred_trgs.append(pred_trg)
        trgs.append([trg])
    return bleu_score(pred_trgs, trgs)
bleu_score = calculate_bleu(test_data, SRC, TRG, model, device)
print(f'BLEU score = {bleu_score*100:.2f}')
# 36.52, which beats the ~34 of the convolutional sequence-to-sequence model and ~28 of the attention based RNN model.

def translate_sentence_vectorized(src_tensor, src_field, trg_field, model, device, max_len=50):
    assert isinstance(src_tensor, torch.Tensor)

    model.eval()
    src_mask = model.make_src_mask(src_tensor)

    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)
    # enc_src = [batch_sz, src_len, hid_dim]

    trg_indexes = [[trg_field.vocab.stoi[trg_field.init_token]] for _ in range(len(src_tensor))]
    # Even though some examples might have been completed by producing a <eos> token
    # we still need to feed them through the model because other are not yet finished
    # and all examples act as a batch. Once every single sentence prediction encounters
    # <eos> token, then we can stop predicting.
    translations_done = [0] * len(src_tensor)
    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).to(device)
        trg_mask = model.make_trg_mask(trg_tensor)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        pred_tokens = output.argmax(2)[:,-1]
        for i, pred_token_i in enumerate(pred_tokens):
            trg_indexes[i].append(pred_token_i)
            if pred_token_i == trg_field.vocab.stoi[trg_field.eos_token]:
                translations_done[i] = 1
        if all(translations_done):
            break

    # Iterate through each predicted example one by one;
    # Cut-off the portion including the after the <eos> token
    pred_sentences = []
    for trg_sentence in trg_indexes:
        pred_sentence = []
        for i in range(1, len(trg_sentence)):
            if trg_sentence[i] == trg_field.vocab.stoi[trg_field.eos_token]:
                break
            pred_sentence.append(trg_field.vocab.itos[trg_sentence[i]])
        pred_sentences.append(pred_sentence)
    return pred_sentences, attention

from torchtext.data.metrics import bleu_score

def calculate_bleu_alt(iterator, src_field, trg_field, model, device, max_len = 50):
    trgs = []
    pred_trgs = []
    with torch.no_grad():
        for batch in iterator:
            src = batch.src
            trg = batch.trg
            _trgs = []
            for sentence in trg:
                tmp = []
                # Start from the first token which skips the <start> token
                for i in sentence[1:]:
                    # Targets are padded. So stop appending as soon as a padding or eos token is encountered
                    if i == trg_field.vocab.stoi[trg_field.eos_token] or i == trg_field.vocab.stoi[trg_field.pad_token]:
                        break
                    tmp.append(trg_field.vocab.itos[i])
                _trgs.append([tmp])
            trgs += _trgs
            pred_trg, _ = translate_sentence_vectorized(src, src_field, trg_field, model, device)
            pred_trgs += pred_trg
    return pred_trgs, trgs, bleu_score(pred_trgs, trgs)


## old

In [ ]:
# Attention Is All You Need https://arxiv.org/pdf/1706.03762.pdf
# https://github.com/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb
# https://colab.research.google.com/github/bentrevett/pytorch-seq2seq/blob/master/6%20-%20Attention%20is%20All%20You%20Need.ipynb
# https://www.mihaileric.com/posts/transformers-attention-in-disguise/
# https://jalammar.github.io/illustrated-transformer/
# http://nlp.seas.harvard.edu/2018/04/03/attention.html

# position embedding <-> "vocabulary" size 100 <-> model can accept sentences up to 100 tokens long
# learned positional encoding, warm-up and cool-down steps, label smoothing

In [ ]:
# @title setup

# https://pytorch.org/tutorials/beginner/translation_transformer.html
# https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c64c91cf87c13c0e83586b8e66e4d74e/translation_transformer.ipynb

# https://github.com/pytorch/data
# %pip install portalocker
# %pip install torchdata

# Create source and target language tokenizer. Make sure to install the dependencies.
!pip install -qU torchdata torchtext
!pip install -qU spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

!git clone --recursive https://github.com/multi30k/dataset.git multi30k-dataset

# !pip list | grep torch
!pip list | grep python

import torchtext.datasets as datasets

# Load the Multi30k dataset
train_iter, valid_iter, test_iter = datasets.Multi30k(split=('train', 'valid', 'test'))

# Iterate through the dataset
for src, tgt in train_iter:
    print(f"Source: {src}")
    print(f"Target: {tgt}")
    break


In [ ]:
# @title data

from torchtext.datasets import multi30k, Multi30k
# modify the URLs for the dataset since the links to the original dataset are broken https://github.com/pytorch/text/issues/1756#issuecomment-1163664163
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TRG_LANGUAGE = 'en'

from torchtext.data.utils import get_tokenizer
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')


UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # unknown, pad, bigining, end of sentence
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

from torchtext.vocab import build_vocab_from_iterator
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))

de_tokens = [de_tokenizer(data_sample[0]) for data_sample in train_iter]
en_tokens = [en_tokenizer(data_sample[1]) for data_sample in train_iter]

de_vocab = build_vocab_from_iterator(de_tokens, min_freq=1, specials=special_symbols, special_first=True)
en_vocab = build_vocab_from_iterator(en_tokens, min_freq=1, specials=special_symbols, special_first=True)
de_vocab.set_default_index(UNK_IDX)
en_vocab.set_default_index(UNK_IDX)

import torch

def de_transform(o):
    o=de_tokenizer(o)
    o=de_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))

def en_transform(o):
    o=en_tokenizer(o)
    o=en_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))


from torch.nn.utils.rnn import pad_sequence
# function to collate data samples into batch tensors
def collate_fn(batch): # convert a batch of raw strings into batch tensors
    src_batch, trg_batch = [], []
    for src_sample, trg_sample in batch:
        src_batch.append(de_transform(src_sample.rstrip("\n")))
        trg_batch.append(en_transform(trg_sample.rstrip("\n")))
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX)
    trg_batch = pad_sequence(trg_batch, batch_first=True, padding_value=PAD_IDX)
    return src_batch, trg_batch


torch.manual_seed(0)

train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
batch_size = 128 # 128
train_loader = torch.utils.data.DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val_iter, batch_size=batch_size, collate_fn=collate_fn)

# vocab_transform = {SRC_LANGUAGE:de_vocab, TRG_LANGUAGE:en_vocab}
# text_transform = {SRC_LANGUAGE:de_transform, TRG_LANGUAGE:en_transform}


In [ ]:
# !pip install -q datasets
import numpy as np
np.float_ = np.float64
np.complex_ = np.complex128
!python -m spacy download de_core_news_sm


In [ ]:
# @title hf data
import torch
from datasets import load_dataset

# Load a translation dataset (e.g., WMT English-German)
dataset = load_dataset("wmt14", "de-en")

# Access train and validation splits
train_data = dataset['train']
val_data = dataset['validation']

# # Example of accessing data
# # for example in train_data:
# #     print(example['translation'])
# batch_size = 128 # 128
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn)
# val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, collate_fn=collate_fn)



# from torchtext.datasets import multi30k, Multi30k
# # modify the URLs for the dataset since the links to the original dataset are broken https://github.com/pytorch/text/issues/1756#issuecomment-1163664163
# multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
# multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TRG_LANGUAGE = 'en'

# from torchtext.data.utils import get_tokenizer
# de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
# en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

# from spacy.tokenizer import Tokenizer
# from spacy.lang.en import English
# nlp = English()
# # Create a blank Tokenizer with just the English vocab
# tokenizer = Tokenizer(nlp.vocab)

# # Construction 2
# from spacy.lang.en import English
# nlp = English()
# tokenizer = nlp.tokenizer

import numpy as np
np.float_ = np.float64
np.complex_ = np.complex128
import spacy
en_tokenizer = spacy.load("en_core_web_sm")
de_tokenizer = spacy.load("de_core_news_sm") # https://spacy.io/models/de



UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # unknown, pad, bigining, end of sentence
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# from torchtext.vocab import build_vocab_from_iterator
# train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))

de_tokens = [de_tokenizer(data_sample[0]) for data_sample in train_iter]
en_tokens = [en_tokenizer(data_sample[1]) for data_sample in train_iter]

# de_vocab = build_vocab_from_iterator(de_tokens, min_freq=1, specials=special_symbols, special_first=True)
# en_vocab = build_vocab_from_iterator(en_tokens, min_freq=1, specials=special_symbols, special_first=True)
# de_vocab.set_default_index(UNK_IDX)
# en_vocab.set_default_index(UNK_IDX)


print("en_tokens", en_tokens)




import torch

def de_transform(o):
    o=de_tokenizer(o)
    o=de_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))

def en_transform(o):
    o=en_tokenizer(o)
    o=en_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))


from torch.nn.utils.rnn import pad_sequence
# function to collate data samples into batch tensors
def collate_fn(batch): # convert a batch of raw strings into batch tensors
    src_batch, trg_batch = [], []
    for src_sample, trg_sample in batch:
        src_batch.append(de_transform(src_sample.rstrip("\n")))
        trg_batch.append(en_transform(trg_sample.rstrip("\n")))
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX)
    trg_batch = pad_sequence(trg_batch, batch_first=True, padding_value=PAD_IDX)
    return src_batch, trg_batch


torch.manual_seed(0)

# train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
# val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
batch_size = 128 # 128
train_loader = torch.utils.data.DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val_iter, batch_size=batch_size, collate_fn=collate_fn)




/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


OSError: [E050] Can't find model 'de_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# @title model
import torch
import torch.nn as nn
import math
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class PositionalEncoder(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_seq_length=512):
        super().__init__()
        self.drop = nn.Dropout(dropout)
        pe = torch.zeros(max_seq_length, d_model)
        pos = torch.arange(0, max_seq_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return self.drop(x + self.pe[:, : x.size(1)])

class LearntPosEnc(nn.Module): # learnt positional embeddings
    def __init__(self, d_model, dropout=0.1, max_length=512):
        super().__init__()
        self.pos_embedding = nn.Embedding(max_length, d_model)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        batch_size, src_len = x.shape
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(device) # [batch size, src len]
        return self.drop(x + self.pos_embedding(pos))


class RoPE(nn.Module): # Rotary Positional Embeddings
    def __init__(self, dim, seq_len=512):
        super().__init__()
        # theta = 1.0 / (10000 ** (torch.arange(0, dim, 2, dtype=torch.float32) / dim))
        theta = 1.0 / (10000 ** (torch.arange(0, dim, 2, dtype=torch.float) / dim))
        # pos = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        pos = torch.arange(seq_len).unsqueeze(1)
        angles = pos * theta # [seq_len, 1] * [dim // 2] = [seq_len, dim // 2]
        self.rot_emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1) # [seq_len, dim]

    def forward(self, x):
        batch, seq_len, dim = x.shape
        # if rot_emb.shape[0] < seq_len: print("rot_emb.shape[0] < seq_len")
        rot_emb = self.rot_emb[:seq_len, :].unsqueeze(0).expand(batch, -1, -1)
        x = x.view(batch, seq_len, dim // 2, 2)
        rot_emb = rot_emb.view(batch, seq_len, dim // 2, 2)
        # rot_x = torch.einsum('...ij,...ij->...ij', x, rot_emb)
        rot_x = x * rot_emb
        # return rot_x.view(*rot_x.shape[:-2], dim)
        return rot_x.flatten(-2)


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads
        self.q = nn.Linear(d_model, d_model, bias=False)
        self.k = nn.Linear(d_model, d_model, bias=False)
        self.v = nn.Linear(d_model, d_model, bias=False)
        self.lin = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.tensor((self.head_dim,), dtype=torch.float, device=device))

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]
        Q = self.q(query).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2)
        K = self.k(key).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2)
        V = self.v(value).view(batch_size, -1, self.n_heads, self.head_dim).transpose(1, 2)
        attn = Q @ K.transpose(2, 3) / self.scale # attn = torch.matmul(Q, K.transpose(2, 3)) / self.scale
        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e10) # [batch, n_heads, seq_len, seq_len]
        attention = torch.softmax(attn, dim=-1)
        x = self.drop(attention) @ V # x = torch.matmul(self.drop(attention), V)
        x = x.transpose(1, 2).reshape(batch_size, -1, self.d_model)
        x = self.lin(x)
        return x, attention


class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.norm1 = nn.RMSNorm(d_model)
        self.norm2 = nn.RMSNorm(d_model)
        self.drop = nn.Dropout(dropout)
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout=0)
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_dim), nn.ReLU(), # ReLU GELU
            nn.Dropout(dropout), nn.Linear(ff_dim, d_model)
        )

    def forward(self, src, src_mask):
        src = self.norm1(src + self.drop(self.self_attn(src, src, src, src_mask)[0]))
        src = self.norm2(src + self.drop(self.ff(src)))
        return src

class Encoder(nn.Module):
    def __init__(self, d_model, n_layers, n_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([EncoderLayer(d_model, n_heads, ff_dim, dropout) for _ in range(n_layers)])

    def forward(self, src, src_mask):
        for layer in self.layers:
            src = layer(src, src_mask)
        return src

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.norm1 = nn.RMSNorm(d_model)
        self.norm2 = nn.RMSNorm(d_model)
        self.norm3 = nn.RMSNorm(d_model)
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout=0)
        self.enc_attn = MultiHeadAttention(d_model, n_heads, dropout=0)
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_dim), nn.ReLU(), # ReLU GELU
            nn.Dropout(dropout), nn.Linear(ff_dim, d_model)
        )
        self.drop = nn.Dropout(dropout)

    def forward(self, trg, enc_src, trg_mask, src_mask):
        trg = self.norm1(trg + self.drop(self.self_attn(trg, trg, trg, trg_mask)[0]))
        trg = self.norm2(trg + self.drop(self.enc_attn(trg, enc_src, enc_src, src_mask)[0]))
        trg = self.norm3(trg + self.drop(self.ff(trg)))
        return trg

class Decoder(nn.Module):
    def __init__(self, d_model, n_layers, n_heads, ff_dim, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(d_model, n_heads, ff_dim, dropout) for _ in range(n_layers)])

    def forward(self, trg, enc_src, trg_mask, src_mask):
        for layer in self.layers:
            trg = layer(trg, enc_src, trg_mask, src_mask)
        return trg

class Seq2Seq(nn.Module):
    def __init__(self, in_dim, out_dim, d_model=512, nhead=8, enc_layers=3, dec_layers=3, ff_dim=512, dropout=0.1):
        super().__init__()
        self.encoder = Encoder(d_model, enc_layers, nhead, ff_dim, dropout)
        self.decoder = Decoder(d_model, dec_layers, nhead, ff_dim, dropout)
        # self.pos_enc = PositionalEncoder(d_model, dropout=dropout)
        # self.pos_enc = LearntPosEnc(d_model, dropout=dropout)
        self.pos_enc = RoPE(d_model)
        self.src_tok_emb = nn.Embedding(in_dim, d_model)
        self.trg_tok_emb = nn.Embedding(out_dim, d_model)
        self.d_model = d_model
        self.lin = nn.Linear(d_model, out_dim)
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, src, trg, src_mask=None, trg_mask=None):
        src = self.pos_enc(self.src_tok_emb(src) * math.sqrt(self.d_model))
        trg = self.pos_enc(self.trg_tok_emb(trg) * math.sqrt(self.d_model))
        enc_src = self.encoder(src, src_mask)
        output = self.decoder(trg, enc_src, trg_mask, src_mask)
        output = self.lin(output)
        return output

    def encode(self, src, src_mask=None):
        return self.encoder(self.pos_enc(self.src_tok_emb(src) * math.sqrt(self.d_model)), src_mask)

    def decode(self, trg, memory, trg_mask=None, src_mask=None):
        trg = self.decoder(self.pos_enc(self.trg_tok_emb(trg) * math.sqrt(self.d_model)), memory, trg_mask, src_mask)
        return self.lin(trg)


in_dim = 50#
out_dim = 50
model = Seq2Seq(in_dim, out_dim, d_model=512, nhead=8, enc_layers=3, dec_layers=3, ff_dim=512, dropout=0.1).to(device)


NameError: name 'de_vocab' is not defined